In [1]:
import pandas as pd
import seaborn as sns

import random

from analisys.backtest import Backtest
from skopt import forest_minimize

In [3]:
class BacktestOptimizer:

    def __init__(self) -> None:
        
        self.backtest_class = Backtest()


    def __call__(self,
                 symbol_x,
                 symbol_y,
                 df):
        
        space = [(1, 4),           #s_spread
                (-4, -1),          #l_spread
                (5, 20),           #period
                (0.01, 0.05),     #tp
                (-0.03, -0.01)]   #sl
        
        def simple_backtest(args):
            
            short_spread, long_spread, period, take_profit, stop_loss = args
            run_backtest = self.backtest_class('BOVA11', 'DOL$', df, 'close', short_spread, 
                                                    long_spread, 10000, period, take_profit, stop_loss, 'roi')
            
            return -run_backtest

        run_optimizer = forest_minimize(simple_backtest, space, random_state=1, verbose=1, n_calls = 20)

        return run_optimizer


In [28]:
import api.metatrader_api as mt5

def get_days(start_date, end_date, d_train, d_test, timeframe = 'D1'):

    #instanciate MetaTrader Server
    mt5_server = mt5.MetaTrader_API(_login=58034070, _password='Ms$12345', _server='XPMT5-DEMO')

    #download market data
    symbols_list = ['BOVA11', 'DOL$']
    hist_list = mt5_server.historical_data_cluster(symbols_list, start_date, end_date, timeframe)

    #match the dates for bova and dol

    hist_list['BOVA11']['date'] = hist_list['BOVA11'].index
    hist_list['DOL$']['date'] = hist_list['DOL$'].index

    hist_list['DOL$'] = hist_list['DOL$'][hist_list['DOL$']['date'].isin(hist_list['BOVA11']['date'])]
    hist_list['BOVA11'] = hist_list['BOVA11'][hist_list['BOVA11']['date'].isin(hist_list['DOL$']['date'])]

    ibov_date = [str(i) for i in list(hist_list['BOVA11']['date'])]

    date_iteration = []
    
    for d in range(len(ibov_date))[:-(d_train+d_test)]:

        train_startdate = ibov_date[d].split(' ')[0]
        train_enddate = ibov_date[d+d_train].split(' ')[0]
        test_startdate = ibov_date[d+d_train].split(' ')[0]
        test_enddate = ibov_date[d+d_test+d_train].split(' ')[0]
            
        date_iteration.append(tuple([train_startdate, train_enddate,
                                    test_startdate, test_enddate]))
        
        real_startdate = date_iteration[0][2]
        real_enddate = date_iteration[-1][-1]

        return date_iteration




A ideia é que vai pegar os dados de date_iteration[0] até date_iteration[1], performar a otimização, e então negociar entre date_iteration[2] e date_iteration[3]


In [29]:
get_days('2023-01-01', '2023-02-01', 5,1)

[('2023-01-02', '2023-01-09', '2023-01-09', '2023-01-10')]